In [6]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
from torch.optim import lr_scheduler

from tqdm.auto import tqdm

In [7]:
train_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(), #隨機將圖片水平翻轉
    transforms.RandomRotation(15), #隨機旋轉圖片
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    
])

In [8]:
batch_size = 32

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("~/data/food-11-big/training", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("~/data/food-11-big/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
test_set = DatasetFolder("~/data/food-11-big/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = 4, pin_memory = True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [9]:
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=11):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512*4*4, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        #print(out.size())
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():
    return ResNet(ResidualBlock)

In [10]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = ResNet18().to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.9, weight_decay=5e-4)
n_epochs = 80
best_acc = 0.0
train_acc_list = []
valid_acc_list = []

for epoch in range(n_epochs):

    model.train()

    train_loss = []
    train_accs = []

    for batch in (train_loader):

        imgs, labels = batch

        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        train_loss.append(loss.item())
        train_accs.append(acc)
        

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    
    train_acc_list.append(float(train_acc))
    
    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    model.eval()

    valid_loss = []
    valid_accs = []

    for batch in (valid_loader):

        imgs, labels = batch

        with torch.no_grad():
          logits = model(imgs.to(device))

        loss = criterion(logits, labels.to(device))

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    valid_acc_list.append(valid_acc)
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    if valid_acc > best_acc:
        torch.save(model.state_dict(), "base_model.pt")
        best_acc = valid_acc

[ Train | 001/080 ] loss = 2.26003, acc = 0.26965
[ Valid | 001/080 ] loss = 2.02232, acc = 0.29365
[ Train | 002/080 ] loss = 1.78556, acc = 0.37965
[ Valid | 002/080 ] loss = 1.79945, acc = 0.37599
[ Train | 003/080 ] loss = 1.66471, acc = 0.42288
[ Valid | 003/080 ] loss = 1.86408, acc = 0.36806
[ Train | 004/080 ] loss = 1.56303, acc = 0.46112
[ Valid | 004/080 ] loss = 1.62990, acc = 0.45000
[ Train | 005/080 ] loss = 1.46024, acc = 0.49830
[ Valid | 005/080 ] loss = 1.66595, acc = 0.43433
[ Train | 006/080 ] loss = 1.38374, acc = 0.52443
[ Valid | 006/080 ] loss = 1.56238, acc = 0.46607
[ Train | 007/080 ] loss = 1.30334, acc = 0.55408
[ Valid | 007/080 ] loss = 1.45745, acc = 0.52073
[ Train | 008/080 ] loss = 1.23630, acc = 0.57974
[ Valid | 008/080 ] loss = 1.43146, acc = 0.52123
[ Train | 009/080 ] loss = 1.16997, acc = 0.59484
[ Valid | 009/080 ] loss = 1.32979, acc = 0.55942
[ Train | 010/080 ] loss = 1.10665, acc = 0.62724
[ Valid | 010/080 ] loss = 1.32638, acc = 0.54603


Exception in thread Thread-125:
Traceback (most recent call last):
  File "/environment/python/versions/3.7.12/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/environment/python/versions/3.7.12/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/environment/python/versions/3.7.12/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/environment/python/versions/3.7.12/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/environment/python/versions/3.7.12/lib/python3.7/site-packages/torch/multiprocessing/reductions.py", line 289, in rebuild_storage_fd
    fd = df.detach()
  File "/environment/python/versions/3.7.12/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  Fil

KeyboardInterrupt: 